# Python Project 2. Война и мир

Текст произведения мы взяли в [библиотеке](http://az.lib.ru/t/tolstoj_lew_nikolaewich/text_0073.shtml) и провели первоначальную обработку. Поскольку наша цель — обработка слов из этого произведения, мы разбили текст на слова и вывели каждое слово в отдельной строке. Кроме того, в местах, где начинаются главы, мы вывели строку [new chapter].

# Загрузка файла

Напишем код, загружающий в память список слов текста

In [ ]:
def read_data():
    data = open('war_peace_processed.txt', 'rt').read().lower()
    return data.split('\n')

Запишем эти данные в переменную:

In [ ]:
data = read_data()

# 1. Подсчет слов

Для начала посчитаем частоту использования отдельных слов в произведении. Для решения задачи воспользуемся уже знакомым нам словарём.

Напишем функцию, считающую слова

In [ ]:
def count_target_word(data,target_word):
    
    frequency = data.count(target_word)

    return frequency

Для проверки работоспособности программы выведем, сколько раз слово "князь" будет найдено в тексте.

In [4]:
count_target_word(data,"князь")

1289

# 2. Подсчет Document Frequency

*Document frequency* (для удобства сократим до *document_frequency*) — это доля документов, в которых встречается искомое слово. В нашем случае речь идёт не о документах, а о главах книги (выше мы писали, что в текстовом документе главы разделяются строкой ```[new chapter]```).

Для начала напишем функцию, разделяющую главы

In [5]:
def split_chapters(data):
    
        lst = []
        a = []
        for j, i in enumerate(data):
            if j + 1 == len(data):
                lst.append(a.copy())
                a.clear()
            elif i != '[new chapter]':
                a.append(i)
            else:
                lst.append(a.copy())
                a.clear()
                
        return lst

Теперь напишем функцию, которая расчитывает *document frequency* для заданного слова ```target_word```

In [6]:
def document_frequency(data, target_word):
    
    lst = split_chapters(data)
    
    number_of_documents = 0
    number_of_documents_with_target_word = 0
    
    for i in lst:
        if target_word in i:
            number_of_documents_with_target_word += 1
        number_of_documents += 1
        
    document_frequency = number_of_documents_with_target_word / number_of_documents
    
    return document_frequency

Посчитаем *document frequency* слова "человек":

In [7]:
document_frequency(data, "человек")

0.672514619883041

# 3. Подсчет Term Frequency

$\textrm{term frequency(слово)} = \dfrac{\textrm{количество раз, когда слово встречается в тексте главы}}{\textrm{количество всех слов в тексте главы}}$

Попробуем посчитать частоту употребления слова *гостья* в $15$-й главе (кстати, у нас главы нумеруются с $0$). Слово гостья встречается в $15$-й главе $10$ раз, а общее количество слов — $1359$.

Итого $\textrm{term frequency(гостья)} \approx 0.007$

Напишем программу:

In [8]:
def term_frequency(data, target_word, target_chapter):
    lst = split_chapters(data)[target_chapter]
    
    frequency = lst.count(target_word)
    term_frequency = frequency / len(lst)
    
    return term_frequency

Посчитаем с помощью программы *term frequency* слова "гостья"

In [9]:
term_frequency(data, "гостья", 15)

0.007358351729212656

# Подсчет контрастности слова

Пришло время дать разъяснения: для чего мы делали вычисления выше и что нас ждет впереди?

Если какое-то слово часто употребляется в документе, то, вероятно, этот документ что-то рассказывает о предмете/действии, описываемом этим словом. Скажем, если вы читаете книгу, в которой много раз употребляется слово *заяц*, то, вероятно, эта книга про зайцев.

Однако, если вы возьмёте слово и, то оно будет встречаться почти в каждой книге много раз. Таким образом, если мы хотим найти наиболее значимые слова в книге, мы, с одной стороны, хотим найти наиболее частые слова, а с другой — убрать те, которые не несут важной информации, так как встречаются везде.

Такая задача хорошо решается с помощью $\textrm{tf*idf}$ — статистической метрики для оценки важности слова в тексте. Другими словами, $\textrm{tf*idf}$ — это «контрастность» слова в документе (насколько оно выделяется среди других слов).

$\textrm{tf*idf} = \textrm{term frequency} * \textrm{inverse document frequency}$

$\textrm{tf}$ — это частотность термина, которая измеряет, насколько часто термин встречается в документе.

$\textrm{idf}$ — это обратная документная частотность термина. Она измеряет непосредственно важность термина во всём множестве документов.

Мы будем использовать не сырые значения $\textrm{tf}$ и $\textrm{idf}$, а их логарифмы, то есть $\log(1+\textrm{tf}) * \log{\textrm{idf}}$.

В качестве примера измерим $\textrm{tf*idf}$ слова анна в главе $4$. Слово анна встречается в указанной главе $7$ раз ($\textrm{tf}$), при этом в $4$ главе $1060$ слов (```chapter_size```), всего же слово анна упоминается в $32$ главах ($\textrm{df}$) из $171$ (```chapter_count```).

Таким образом $\textrm{tf*idf}$ данного слова будет примерно равен $0.01$

Напишем программу:

In [10]:
def tf_idf(data, target_word, target_chapter):
    from math import log
    
    log_tf = log(1 + term_frequency(data,target_word, target_chapter))
    log_idf = log(1 / document_frequency(data, target_word))
    
    return  log_tf*log_idf

Проверим работу программы на слове "анна"

In [11]:
tf_idf(data, "анна", 4)

0.011031063403921838

# 5. Топ 3 слов

Теперь, когда мы умеем вычислять $\textrm{tf*idf}$ для каждого слова в главе, мы можем найти те слова, которые являются самыми «контрастными» для данной главы, то есть они могут являться в своём роде заголовком для главы.

Напишем программу:

In [12]:
def top3(data, target_chapter):
    lst = split_chapters(data)
    data_set = set(lst[target_chapter])
    
    top = [[i,tf_idf(data, i, target_chapter)] for i in data_set]
    
    top3 = sorted(top,key=lambda t:t[1])[:-4:-1]
        
    return top3

Посмотрим на топ 3 слова четвертой главы:

In [13]:
top3(data, 4)

[['павловна', 0.014125754682156953],
 ['анна', 0.011031063403921838],
 ['прядильной', 0.00969211136471989]]

Таким образом можно предположить, что речь в главе больше всех фигурирует Анна Павловна, и, если прочитать эту главу, то все действительно так.

# Итого

По итогам этого проекта был получен опыт работы с такой статистической мерой, как $\textrm{TF-IDF}$, которая позволяет сравнивать и исследовать тексты.